Единственная задача: обобщить написанный на семинаре код и сымитировать работу одного большого отдела ABBYY, в котором есть три маленьких подотдела с лингвистами, программистами и комплингом. То есть, что у нас должно быть реализовано:

- родительский класс "работник"
- базовые классы "лингвист", "программист" и "компьютерный лингвист"
- у всех методы work
- классы "босс_лингвист", "босс_программист" и "босс_кл", которые могут наследовать (с подмешиванием) от общего класса "босс"
- у боссов в атрибутах сидят их подчиненные
- босс подотдела получает квесты от менеджера главного отдела и принуждает сотрудников работать
- в главном отделе есть метод для выдачи квестов
- соответственно, используем как наследование, так и композицию с делегированием

In [1]:
from time import sleep

In [2]:
class Worker:
    '''Общий класс-родитель (дедушка)'''
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        self.__salary = 350  
        self.bankaccount = 0

    def work(self, *args):
        raise NotImplementedError
    

class Linguist(Worker):
    '''Класс-мама'''
    def __init__(self, name, surname):
        Worker.__init__(self, name, surname)
        self.__salary = self._Worker__salary * 1.5
        self.publications = []
    
    def work(self, hours, name_of_publ):
        print('Working...')
        sleep(hours)
        self.bankaccount += self.__salary * hours
        self.publications.append(name_of_publ)
        print(f'{self.name} {self.surname} has published a paper {name_of_publ} in Voprosy Yazykoznaniya')

    def readlingpapers(self, lingpaper):
        print('Reading...')
        sleep(len(lingpaper) // 10)
        print(f'{self.name} {self.surname} has read {lingpaper}')


class Programmer(Worker):
    '''Класс-папа'''
    def __init__(self, name, surname):
        Worker.__init__(self, name, surname)
        self.__salary = self._Worker__salary * 2.5
        self.projects = []

    def work(self, hours, name_of_project):
        print('Working...')
        sleep(hours)
        self.bankaccount += self.__salary * hours
        self.projects.append(name_of_project)
        print(f'{self.name} {self.surname} has committed a repo {name_of_project} to GitHub')

    def learnpython(self, hours):
        print('Studying...')
        sleep(hours)
        print(f'{self.name} {self.surname} has learned a bit of Python')


class ComputerLinguist(Linguist, Programmer):
    '''Наследуемся одновременно от лингвиста и от программиста'''
    def __init__(self, name, surname):
        Linguist.__init__(self, name, surname)  
        Programmer.__init__(self, name, surname)
        self.salary = self._Programmer__salary * 0.9

    def work(self, hours, name_of_paper_with_code):
        print('Working...')
        sleep(hours)
        self.publications.append(name_of_paper_with_code)
        self.projects.append(name_of_paper_with_code)
        self.bankaccount += self.salary * hours
        print(f'{self.name} {self.surname} has published {name_of_paper_with_code} in ACL Papers')


class Boss:
    def __init__(self, name, surname, employees=None):
        self.name = name
        self.surname = surname
        self.employees = employees if employees else []
    
    def receive_task(self, task):
        raise NotImplementedError('метод receive_task в наследниках')
    
    def manage(self, task): # заставляет работать
        print(f'Босс {self.name} дает задачи своим сотрудникам.')
        for employee in self.employees:
            employee.work(1, task)


class LingBoss(Boss, Linguist):
    def __init__(self, name, surname, employees=None):
        super().__init__(name, surname, employees)
        
    def receive_task(self, task):
        print(f'Босс лингвистов {self.name} получил задачу: {task}')
        self.manage(task)


class ProgBoss(Boss, Programmer):
    def __init__(self, name, surname, employees=None):
        super().__init__(name, surname, employees) 
        
    def receive_task(self, task):
        print(f'Босс программистов {self.name} получил задачу: {task}')
        self.manage(task)


class CompLingBoss(Boss, ComputerLinguist):
    def __init__(self, name, surname, employees=None):
        super().__init__(name, surname, employees)
        
    def receive_task(self, task):
        print(f'CL босс {self.name} получил задачу: {task}')
        self.manage(task)


class MainDepartment:
    def __init__(self, bosses, tasks):
        self.bosses = bosses #[]
        self.tasks = tasks

    def give_tasks(self):
        for boss, task in zip(self.bosses, self.tasks):
            boss.receive_task(task)


In [3]:
if __name__ == "__main__":

    linguist = Linguist('Natasha', 'Sidorova')
    linguist1 = Linguist('Alex', 'Vorobyev')
    programmer = Programmer('Steve', 'Jobs')
    programmer1 = Programmer('Guido', 'van Rossum')
    compling = ComputerLinguist('Petya', 'Ivanov')
    compling1 = ComputerLinguist('Lida', 'Petrova')

    bossling = LingBoss('Linguistics', 'Boss', [linguist, linguist1])
    bossprog = ProgBoss('Programming', 'Boss', [programmer, programmer1])
    clboss = CompLingBoss('CompLing', 'Boss', [compling, compling1])

    tasks = ['Статья A', 'Программа для разметки Y', 'Создание датасета для решения задачи Х']
    main_department = MainDepartment([bossling, bossprog, clboss], tasks)
   
    main_department.give_tasks()

Босс лингвистов Linguistics получил задачу: Статья A
Босс Linguistics дает задачи своим сотрудникам.
Working...
Natasha Sidorova has published a paper Статья A in Voprosy Yazykoznaniya
Working...
Alex Vorobyev has published a paper Статья A in Voprosy Yazykoznaniya
Босс программистов Programming получил задачу: Программа для разметки Y
Босс Programming дает задачи своим сотрудникам.
Working...
Steve Jobs has committed a repo Программа для разметки Y to GitHub
Working...
Guido van Rossum has committed a repo Программа для разметки Y to GitHub
CL босс CompLing получил задачу: Создание датасета для решения задачи Х
Босс CompLing дает задачи своим сотрудникам.
Working...
Petya Ivanov has published Создание датасета для решения задачи Х in ACL Papers
Working...
Lida Petrova has published Создание датасета для решения задачи Х in ACL Papers
